In [1]:
import os,sys
sys.path.append(os.path.abspath("../"))
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
from models.model import VisionModel, LanguageModel, EncoderDecoder
from transformers import GPT2Config, GPT2Tokenizer
vm_name = "laion/CLIP-ViT-L-14-laion2B-s32B-b82K" #"openai/clip-vit-large-patch14" 
lm_name = "gpt2-medium" #"Gustavosta/MagicPrompt-Stable-Diffusion" 

config = GPT2Config.from_pretrained(lm_name)
tokenizer = GPT2Tokenizer.from_pretrained(lm_name)
n_image_token = 4
lm = LanguageModel.from_pretrained(lm_name, config=config)
vm = VisionModel(model_name=vm_name, out_features=config.n_embd* n_image_token)

#tokenizer.add_special_tokens({"pad_token": "[PAD]"})
tokenizer.pad_token = tokenizer.eos_token

tokenizer.add_tokens(["<IMG>"])
# Resize the model's token embeddings to account for the new token(s)
lm.resize_token_embeddings(len(tokenizer))

/home/jaehoon_scatterlab_co_kr/projects/image-to-prompts/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at laion/CLIP-ViT-L-14-laion2B-s32B-b82K were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.1.self_attn.out_proj.weight', 'text_model.encoder.layers.3.self_attn.out_proj.weight', 'text_model.encoder.layers.5.self_attn.k_proj.bias', 'visual_projection.weight', 'text_model.encoder.layers.5.self_attn.v_proj.bias', 'text_model.encoder.layers.9.self_attn.out_proj.bias', 'text_model.encoder.layers.9.self_attn.k_proj.weight', 'text_model.encoder.layers.7.self_attn.k_proj.bias', 'text_model.encoder.layers.9.self_attn.q_proj.bias', 'text_model.encoder.layers.3.self_attn.k_proj.bias', 'text_model.encoder.layers.2.self_attn.v_proj.weight', 'text

Embedding(50258, 1024)

In [3]:
import torch
model = EncoderDecoder(
        encoder=vm,
        decoder=lm,
        config={
            "encoder_frozen": True,
            "decoder_frozen": True,
        }
    )
model.eval()

EncoderDecoder(
  (encoder): VisionModel(
    (model): CLIPVisionModel(
      (vision_model): CLIPVisionTransformer(
        (embeddings): CLIPVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
          (position_embedding): Embedding(257, 1024)
        )
        (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (encoder): CLIPEncoder(
          (layers): ModuleList(
            (0): CLIPEncoderLayer(
              (self_attn): CLIPAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
              )
              (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
              (mlp): CLIPMLP(
    

In [4]:
ckpt = torch.load('../results/4-laion-right-0411-v6/best-model.ckpt')
model.load_state_dict(ckpt)

<All keys matched successfully>

In [5]:
import cv2
IMAGENET_MEAN_RGB = [0.485, 0.456, 0.406]
IMAGENET_STD_RGB = [0.229, 0.224, 0.225]

def process(path, image_size=(224,224)):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size, interpolation=cv2.INTER_AREA)
    image = image / 255.0
    image = (image - IMAGENET_MEAN_RGB) / IMAGENET_STD_RGB
    image = image.transpose(2, 0, 1)
    pixel_values = torch.tensor(image, dtype=torch.float)

    return pixel_values

In [6]:
flist = os.listdir('../datasets/test/')
bad_word_tokens = [tokenizer.encode('<IMG>', add_prefix_space=True)]
for f in flist:
    pixel_values = process('../datasets/test/'+f)
    pixel_values = pixel_values.unsqueeze(0)
    out = model.generate(input_ids=None, pixel_values=pixel_values, max_length=32, do_sample=True, top_p=0.1, no_repeat_ngram_size=3, repetition_penalty=1.2, pad_token_id=tokenizer.eos_token_id)
    out_sequence = tokenizer.decode(out.sequences.tolist()[0], skip_special_tokens=True)
    print(f)
    print(out_sequence)
    

a4e1c55a9.png
a robot with a black and white drawing on the front, an inkjet printer inked charcoal print. origami style drawings of robots drawn by robert
227ef0887.png
a wood carving of a wolf, carved with an intricate spiral pattern and decorated by the shape-shifting artistry in nature. handmade wooden ornament for jewelry
c98f79f71.png
a portrait of a man with an eye patch, oil painting by the artist Raphael Antonovitch. Oil on canvas and acrylic paint in black & white style
f27825b2c.png
a man in a blue shirt eating at the restaurant, with his hands on top of him and an iced tea cup.
The manager is yelling "
92e911621.png
a dinosaur eating a cheese in the forest, with an orange shell and green leaves. art by jordan kirkpatrick on DeviantArt]
The
20057f34d.png
a giant hole in the earth, a rift is forming behind it. high quality art style and animation by kyle_dellacosta on photoshop
d8edf2e40.png
a cherry blossom in a forest, with the sun shining on it. shot of an astronaut walkin

In [7]:
out.sequences

tensor([[50256,    64,   582,   287,   257, 36701,  1097,    11,  2263,   572,
           351,   465,  3290,   319,   262,  2975,   290,  6155,   832,   281,
         23978,    13,   357,  8929,  6323,     8,   198, 50257,     1,  2825,
          3768, 38333,  2939,   286,   256, 15566,  2836,   366,    74,   313,
          8719,   259,   261,  6277,    62,    73,  2674,  2771,  5320,  4023,
         50257,   464,   995,   318,  1336,    12,  4421,   306,   925,   510,
           416,   477,   428,  4998,  1242,     0,   383,   749,  4950,  1517,
           546,   340,    30,   632,   338,   407,   772,  1969]])

In [47]:
pixel_values = process('../datasets/test/227ef0887.png')
pixel_values = pixel_values.unsqueeze(0)

In [48]:
inputs = tokenizer("<IMG>", return_tensors='pt')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
out = model.generate(input_ids=None, pixel_values=pixel_values, max_length=78, do_sample=True, top_p=0.1, no_repeat_ngram_size=3, repetition_penalty=1.2, pad_token_id=tokenizer.eos_token_id)

In [49]:
out.sequences

tensor([[50256,    64,  4950,  4898, 23470, 13510, 39510,   286,   257,  5509,
            11,   416,   308, 10316, 26924,   290,   281, 28746,  3912,   925,
           503, 26929,  6147,   351,   262,  5485, 11112,  1486,   319,  3348,
          1242,   287,  5405,    13,   198, 50256]])

In [50]:
tokenizer.decode(out.sequences.tolist()[0], skip_special_tokens=True)

'a beautiful wood carved wooden carving of a tree, by greek sculpture and an intricate pattern made out curved metal with the shape logo design on paper art in glass.\n'

In [31]:
image_embeddings = vm(pixel_values=pixel_values)

In [32]:
image_embeddings.shape

torch.Size([1, 768])

In [33]:
import sys
import os
sys.path.append(os.path.abspath('../input/sentence-transformers'))
from sentence_transformers import SentenceTransformer
st_model = SentenceTransformer('../input/all-MiniLM-L6-v2')

In [34]:
out = st_model.encode(['hi','ja'])
out.shape

(2, 384)

In [37]:
tokenizer.convert_tokens_to_ids('<IMG>')

50258

In [25]:
tokenizer.convert_ids_to_tokens([0])

['!']

In [39]:
tokenizer.pad_token_id

50257

In [8]:
from models.dataset import ImageTextDataset
import json
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion")

tokenizer.add_special_tokens({"pad_token": "[PAD]"})
tokenizer.add_tokens(["<IMG>"])

validation_data_path = "../datasets/validation/metadata.jsonl"
with open(validation_data_path) as f:
    validation_data = [json.loads(d) for d in f.readlines()]
validation_dataset = ImageTextDataset(
    validation_data,
    tokenizer,
    max_seq_length=77,
    image_size=(224,224),
    root_path="/home/jaehoon_scatterlab_co_kr/projects/image-to-prompts/datasets/validation",
)

In [17]:
data = validation_dataset[-19]

In [18]:
data

{'input_ids': tensor([50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
         50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
         50257, 50257, 50257, 50257, 50258, 50256,  6214,   262,  1200,    13,
           679,   318, 14005,   290,  7888,    11,   339, 17326,   257,  7888,
           290,   374, 14655, 41822, 10147,    13,   679,   336,  3369,   262,
           629,   724,  1924,  2046,    13, 22151,  6486,  3223,  2900,  7032,
           351,   374,  3775,   286,  6729,   290, 18646, 16479,  3675,   326,
         25451,  1865,   257,  1178,   938, 23214, 50256]),
 'attention_mask': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]),
 'image_token_mask': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [16]:
validation_data[-19]

{'file_name': '00132475.jpg',
 'text': 'See the child. He is pale and thin, he wears a thin and ragged linen shirt. He stokes the scullery fire. Outside lie dark turned fields with rags of snow and darker woods beyond that harbor yet a few last wolves'}

In [19]:
len(validation_dataset)

8798

In [21]:
8798%32

30

In [32]:
tokenizer.pad_token = tokenizer.eos_token

In [33]:
tokenizer.pad_token_id

50256

In [62]:
lm.config.n_embd

768

In [65]:
tokenizer.unk_token

'<|endoftext|>'

In [18]:
import json
with open('../datasets/train/metadata.jsonl') as f:
    data = [json.loads(d) for d in f.readlines()]

In [19]:
cnt = 0
for t in data:
    if 'woman' in t['text']:
        cnt +=1
print(cnt)

5690


In [20]:
len(data)

254146

In [29]:
for t in data:
    if 'greg' in t['text']:
        print(t['text'])
        break

abstract watercolor painting of hedonic scandinavian king house, in stone, straw roof, war lord, magical and traditional, runes, cinematic light, national romanticism by anders zorn, by greg rutkowski, by greg manchess


In [40]:
import re

def find_three_words(text):
    pattern = r'by(\s+\b\w+\b){1,3}'
    matches = re.findall(pattern, text)
    return matches

res = []
for idx,t in enumerate(data):
    text = t['text']
    results = find_three_words(text)

    for match in results:
        words = match.strip().split()
        #print(words)
        res.extend(words)

In [41]:
len(res)

57053

In [43]:
import collections
c = collections.Counter(res)

In [46]:
c.most_common(50)

[('and', 5518),
 ('rutkowski', 984),
 ('of', 569),
 ('artgerm', 443),
 ('greg', 442),
 ('in', 440),
 ('a', 371),
 ('wlop', 300),
 ('beksinski', 277),
 ('the', 260),
 ('by', 258),
 ('moebius', 246),
 ('with', 240),
 ('jean', 216),
 ('Rutkowski', 206),
 ('ghibli', 200),
 ('bussiere', 198),
 ('mucha', 184),
 ('beeple', 182),
 ('rembrandt', 181),
 ('on', 173),
 ('Lau', 167),
 ('gogh', 165),
 ('shinkai', 158),
 ('frazetta', 153),
 ('bosch', 153),
 ('giger', 152),
 ('h', 150),
 ('dali', 148),
 ('vinci', 143),
 ('artist', 139),
 ('monet', 138),
 ('rhads', 134),
 ('H', 125),
 ('rockwell', 123),
 ('Rembrandt', 117),
 ('tran', 114),
 ('mohrbacher', 108),
 ('Gogh', 107),
 ('stalenhag', 106),
 ('art', 104),
 ('Vinci', 100),
 ('hopper', 99),
 ('ito', 98),
 ('Beksiński', 98),
 ('ejsing', 97),
 ('ross', 97),
 ('caravaggio', 97),
 ('Mucha', 97),
 ('gurney', 97)]

In [45]:
import collections
c = collections.Counter()
for t in data:
    words = t['text'].split(' ')
    for word in words:
        c[word] +=1

In [48]:
c.most_common(50)

[('a', 168214),
 ('of', 117810),
 ('the', 92853),
 ('in', 80496),
 ('and', 55053),
 ('by', 55000),
 ('with', 42057),
 ('on', 41593),
 ('art', 19390),
 ('an', 19007),
 ('from', 17751),
 ('style', 16235),
 ('as', 15752),
 ('photo', 14530),
 ('painting', 13762),
 ('detailed', 12983),
 ('at', 12969),
 ('detailed,', 12478),
 ('art,', 12203),
 ('digital', 11585),
 ('to', 10825),
 ('highly', 10413),
 ('is', 10075),
 ('high', 9953),
 ('A', 9821),
 ('trending', 9779),
 ('man', 8552),
 ('artstation,', 8215),
 ('beautiful', 8211),
 ('wearing', 8170),
 ('black', 8156),
 ('white', 7641),
 ('realistic', 7527),
 ('concept', 7358),
 ('lighting,', 7241),
 ('made', 6782),
 ('artstation', 6764),
 ('for', 6381),
 ('his', 6354),
 ('cinematic', 5936),
 ('dark', 5847),
 ('still', 5728),
 ('painting,', 5558),
 ('movie', 5543),
 ('oil', 5412),
 ('red', 5359),
 ('photograph', 5248),
 ('realistic,', 5197),
 ('very', 5025),
 ('octane', 4876)]

In [1]:
import json
with open('../datasets/train/metadata.jsonl') as f:
    data = [json.loads(d) for d in f.readlines()]

In [6]:
from tqdm import tqdm
import collections
c = collections.Counter()
texts = [d['text'] for d in data]
for t in tqdm(texts):
    t = t.replace(',','')
    words = t.split(' ')
    for i in range(len(words)-1):
        bigram = (words[i], words[i+1])
        c[bigram] +=1

100%|██████████| 254146/254146 [00:04<00:00, 62426.16it/s]


In [7]:
c.most_common(100)

[(('of', 'a'), 28357),
 (('in', 'the'), 27638),
 (('in', 'a'), 17114),
 (('of', 'the'), 11199),
 (('style', 'of'), 10933),
 (('highly', 'detailed'), 9592),
 (('with', 'a'), 9539),
 (('on', 'a'), 9154),
 (('the', 'style'), 9132),
 (('trending', 'on'), 9069),
 (('photo', 'of'), 7757),
 (('on', 'artstation'), 7143),
 (('on', 'the'), 6967),
 (('digital', 'art'), 6870),
 (('concept', 'art'), 6372),
 (('as', 'a'), 6021),
 (('painting', 'of'), 5843),
 (('art', 'by'), 4758),
 (('from', 'the'), 4363),
 (('wearing', 'a'), 4238),
 (('octane', 'render'), 4098),
 (('made', 'of'), 3830),
 (('of', 'an'), 3813),
 (('at', 'the'), 3481),
 (('oil', 'painting'), 3377),
 (('painting', 'by'), 3373),
 (('sharp', 'focus'), 3147),
 (('greg', 'rutkowski'), 3017),
 (('high', 'quality'), 3009),
 (('and', 'a'), 3002),
 (('photograph', 'of'), 2989),
 (('out', 'of'), 2907),
 (('3', 'd'), 2857),
 (('holding', 'a'), 2775),
 (('a', 'man'), 2771),
 (('a', 'beautiful'), 2668),
 (('at', 'a'), 2612),
 (('digital', 'paintin

In [10]:
res = []
for t in texts:
    res.append(t.count(','))
c = collections.Counter(res)
c.most_common(50)

[(0, 114338),
 (1, 43976),
 (2, 29298),
 (3, 20766),
 (4, 13826),
 (5, 9281),
 (6, 7219),
 (7, 4511),
 (8, 3230),
 (9, 2314),
 (10, 1710),
 (11, 1217),
 (12, 963),
 (13, 571),
 (14, 397),
 (15, 242),
 (16, 135),
 (17, 51),
 (18, 32),
 (19, 30),
 (20, 13),
 (21, 10),
 (22, 10),
 (24, 3),
 (23, 2),
 (35, 1)]

In [9]:
'asf, asdf, '.count(',')

2

In [12]:
res = {}
for t in texts:
    cnt = t.count(',')
    if cnt >8:
        if cnt not in res:
            res[cnt] = [t]
        else:
            res[cnt].append(t)


In [15]:
res[14][:10]

['the sacred clown, realistic, extremely detailed, sharp focus, wide view, smooth, digital illustration, by james jean, by rossdraws, frank franzzeta, mcbess, sakimichan, brosmin, danton fadeev, steve simpson',
 'gorgeous eva green, evil demon, full body shot, torn black dress, evil smile, spooky, illustration, symmetrical face realistic symmetrical eyes, cinematic lighting, artgerm, hyperdetailed, charlie bowater, beautiful, elegant, dark bedroom marc simonetti',
 'a scene in beach, concept art, hyperrealistic, ultra realistic, trending on artstation, high quality, advanced technology, highly detailed, path traced, soft lighting, video game, soft colors, digital art, octane render, 5',
 'small traffic road, under large traffic road, futuristic transportation, cyberpunk, extremely detailed, top view, digital illustration, by artstation, artfol, artslant artbox, cgsociety, gfxartist, sakimichan, mcbess, aphonse mucha',
 'light souls by fromsoftware, cheerful, healing, spiritual, colorfu

In [14]:
tokenizer.convert_ids_to_tokens([0,1,2,3,4,5,6,7,8,9,10,11])

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',']

In [18]:
tokenizer.convert_ids_to_tokens([50254,50255,50256])

['Ġinformants', 'Ġgazed', '<|endoftext|>']